In [1]:
import pandas as pd

In [ ]:
rename_columns = {
    "Data_ID": "dataset_id",
    "IndicatorID": "indicator_id",
    "Category": "category",
    "Indicator Name": "name",
    "Data_Name": "name",
    "Indicator_Description": "description",
    "Indicator Source(s)": "source",
    "General_Description_Datasets": "description",
    "Dataset_Examples_ Citation": "citation",
    "Dataset_Sources_Examples_ Licenses": "licenses",
    "Dataset_Examples_ Notes": "notes",
}

# ddf = pd.read_csv("datasets.csv").dropna(how="all").rename(columns=rename_columns).set_index("dataset_id")
# idf = pd.read_csv("indicators.csv").dropna(how="all").rename(columns=rename_columns).set_index("indicator_id")

idf = (
    pd.read_excel("catalog.xlsx", sheet_name="Page_1_CIC_Indicators_Data")
    .dropna(how="all")
    .rename(columns=rename_columns)
    .set_index("indicator_id")
)
ddf = (
    pd.read_excel("catalog.xlsx", sheet_name="Page_2_Data_To_Indicators")
    .dropna(how="all")
    .rename(columns=rename_columns)
    .set_index("dataset_id")
)
# ddf.columns

In [ ]:
link_cols = [c for c in ddf.columns if c.startswith("Ind") and c.endswith("ID")]
# ddf[[c for c in ddf.columns if c not in link_cols]].to_csv("datasets.csv", index=True)
ddf_links = ddf[link_cols]

link_cols = [c for c in idf.columns if c.startswith("Data_") and c.endswith("ID")]
# idf[[c for c in idf.columns if c not in link_cols]].to_csv("indicators.csv", index=True)
idf_links = idf[link_cols]

In [ ]:
idf_links.reset_index().melt(
    id_vars=["indicator_id"],
    value_vars=link_cols,
    value_name="dataset_id",
).replace("xxx", pd.NA).dropna(subset=["dataset_id"]).drop(
    columns=["variable"]
).sort_values(
    by=["indicator_id", "dataset_id"]
).to_csv(
    "links.csv", index=False
)

In [ ]:
ddf[[c for c in ddf.columns if c in rename_columns.values()]].to_csv(
    "datasets.csv", index=True
)
idf[[c for c in idf.columns if c in rename_columns.values()]].to_csv(
    "indicators.csv", index=True
)

In [ ]:
# group links for airtable
links = pd.read_csv("links.csv")
links = (
    links.groupby("indicator_id")["dataset_id"]
    .apply(lambda x: ",".join(x))
    .reset_index()
)
links.to_csv("links_grouped.csv", index=False)